## Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data. The torch.nn namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

C:\Users\Predator\Envs\kn_ml\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Predator\Envs\kn_ml\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


## Get Device for Training

We want to be able to train our model on a hardware accelerator like the GPU, if it is available.

In [4]:
torch.cuda.is_available()

True

In [6]:
device='cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device.')

Using cuda device.


## Define the Class

We define our neural network by subclassing nn.Module, and initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements the operations on input data in the `forward` method.

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of NeuralNetwork, and move it to the device, and print its structure.

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s `forward`, along with some background operations.

**Note: Do not call model.forward() directly!**

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class. We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.

In [10]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5], device='cuda:0')


## Model Layers

Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [11]:
input_image = torch.rand(3,28,28)
print(f'Input Image Size: {input_image.size()}')

Input Image Size: torch.Size([3, 28, 28])


### nn.Flatten

We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [12]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(f'Flattened Image Size: {flat_image.size()}')

Flattened Image Size: torch.Size([3, 784])


### nn.Linear
The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [13]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(f'Size: {hidden1.size()}')

Size: torch.Size([3, 20])


### nn.ReLU
Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use nn.ReLU between our linear layers, but there’s other activations to introduce non-linearity in your model.

In [14]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.8319, -0.1721, -0.0716,  0.2398, -0.0269,  0.2940, -0.5291,  0.1226,
          0.3591, -0.7268, -0.0308,  0.0095, -0.0110, -0.1148,  0.2354,  0.1643,
          0.4428,  0.1819, -0.4861,  0.1722],
        [-0.5762, -0.0754,  0.1635,  0.0666, -0.0271, -0.0924, -0.7892, -0.0913,
          0.3078, -0.6046,  0.1222, -0.3129,  0.0489,  0.0698,  0.2917, -0.2164,
          0.3277,  0.0205, -0.5360,  0.4001],
        [-0.5312, -0.3226,  0.0546,  0.1452, -0.3331,  0.2685, -0.9305,  0.0301,
          0.3463, -0.6373, -0.2331, -0.3579,  0.2977,  0.1000,  0.2920,  0.1097,
          0.5164,  0.2044, -0.1971,  0.6601]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.2398, 0.0000, 0.2940, 0.0000, 0.1226, 0.3591,
         0.0000, 0.0000, 0.0095, 0.0000, 0.0000, 0.2354, 0.1643, 0.4428, 0.1819,
         0.0000, 0.1722],
        [0.0000, 0.0000, 0.1635, 0.0666, 0.0000, 0.0000, 0.0000, 0.0000, 0.3078,
         0.0000, 0.1222, 0.0000, 0.0489, 0.0698, 0.29

### nn.Sequential
nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like seq_modules.

In [15]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax
The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters

Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s `parameters()` or `named_parameters()` methods.

In [17]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0089,  0.0029, -0.0045,  ..., -0.0275,  0.0277,  0.0213],
        [ 0.0141,  0.0046,  0.0216,  ..., -0.0208,  0.0268,  0.0117]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0198, 0.0203], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0031, -0.0284, -0.0407,  ..., -0.0064, -0.0440, -0.0264],
        [-0.0347,  0.0248,  0.0120,  ..., -0.0090,  0.0353,  0.0427]],
       device='cuda:0', grad_fn=<Sl